In [38]:
import preprocessing as preproc
from utils import *
import collections

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [66]:
class MetricsData(object):

    def __init__(self,word,appearances,metrics_dict):
        self.word = word
        self.appearances = appearances
        self.metrics_dict = metrics_dict
        

In [72]:
def status((groundTruthValue,predictedValue)):
    if(groundTruthValue):
        if(predictedValue):
            return "TP"
        return "FN"
    
    if(predictedValue):
        return "FP"
    
    return "TN"

def get_metrics(true_word_mask,predicted_word_mask):
    truth2predictedArr = zip(true_word_mask,predicted_word_mask)
    statusArr = [status(truth2predicted) for truth2predicted in truth2predictedArr]
    counter = collections.defaultdict(int,collections.Counter(statusArr).most_common())
    
    tp = counter['TP']
    fp = counter['FP']
    tn = counter['TN']
    fn = counter['FN']
    
    precision = float(tp)/max(1,(tp+fp))
    recall = float(tp)/max(1,(tp+fn))

    return {"precision":precision,"recall":recall}

def containsWord(sentence, word):
    return word in sentence

def get_word_mask(captions,word):
    return [containsWord(caption, word) for caption in captions]

def computeMetricsData(word):
    true_word_mask = get_word_mask(true_captions,metric_word)
    predicted_word_mask = get_word_mask(predicted_captions,metric_word)

    nr_instances = sum(true_word_mask)
    metrics_dict = get_metrics(true_word_mask,predicted_word_mask)

    return MetricsData(metric_word,nr_instances,metrics_dict)

def displayMetrics(metric_data_arr):
    
    for metric_data in metric_data_arr:
       
        print(metric_data.word+" => "+str(metric_data.appearances)+" instances")
        print("\tPrecision: "+str(metric_data.metrics_dict["precision"]))
        print("\tRecall: "+str(metric_data.metrics_dict["recall"]))
        print("")


In [56]:
base_path = app_3_length_15_data_path

train_path = base_path + train_folder
val_path = base_path + val_folder

test_indexed_captions_path = base_path + val_folder + batch_folder + indexed_captions_folder
test_raw_captions_path = base_path + val_folder + batch_folder+captions_folder


In [62]:
predicted_captions_raw = preproc.load_obj(val_path+predictions_folder+"predicted_captions")
predicted_captions = [caption.split() for caption in predicted_captions_raw]

size = len(predicted_captions)

(test_captions_raw,_) = preproc.get_captions_raw_and_indexed(test_raw_captions_path,test_indexed_captions_path)
true_captions = [caption[0].split() for caption in test_captions_raw[:size]]


print(len(true_captions))
print(len(predicted_captions))


100%|██████████| 14/14 [00:00<00:00, 453.66it/s]

128
128


In [63]:
metric_words = ['tennis','snow','train','cat','red','yellow','food','kitchen']

In [68]:
metric_data_arr = [computeMetricsData(metric_word) for metric_word in metric_words]

In [73]:
displayMetrics(metric_data_arr)

tennis => 3 instances
	Precision: 0.75
	Recall: 1.0

snow => 3 instances
	Precision: 0.666666666667
	Recall: 0.666666666667

train => 4 instances
	Precision: 0.75
	Recall: 0.75

cat => 3 instances
	Precision: 0.75
	Recall: 1.0

red => 5 instances
	Precision: 0.0
	Recall: 0.0

yellow => 3 instances
	Precision: 0.0
	Recall: 0.0

food => 4 instances
	Precision: 0.25
	Recall: 0.25

kitchen => 3 instances
	Precision: 0.0
	Recall: 0.0



In [ ]:
displayAggMetrics(metric_data_arr)

In [ ]:
def displayAggMetrics(metric_data_arr):
    total_instances = sum([metric_data.appearances for metric_data in metric_data_arr])
    
    
    for key in metric_data_arr[0].metrics_dict.iteritems():
        metric_sum = sum([metric_data.appearances*metric_data.metrics_dict[key] for metric_data in metric_data_arr])        
        weighted_metric = float(precision_sum)/total_instances
        print("Weighted "+str(key)+str(": ") + str"weighted_precision)
    print(weighted_recall)

28

0.375
0.428571428571


In [47]:
def most_common_words(captions,word_limit = None):
    
    words = []
    for caption in captions:
        for word in caption:
            words.append(word)
        
    common_words2app = None
    if(word_limit is None):
        common_words2app = collections.Counter(words).most_common()
    else:
        common_words2app = collections.Counter(words).most_common(word_limit)
        
    common_words2app = [(word,app) for word,app in common_words2app if word.lower() not in stopwords.words('english')]
    common_words2app = [(word,app) for word,app in common_words2app if word not in ['START','END']]

    return common_words2app

In [51]:
common_words2app = most_common_words(true_captions)

In [52]:
common_words2app

[(u'man', 3573),
 (u'sitting', 3300),
 (u'standing', 2625),
 (u'next', 2441),
 (u'white', 2001),
 (u'people', 1779),
 (u'top', 1714),
 (u'holding', 1654),
 (u'woman', 1598),
 (u'riding', 1367),
 (u'street', 1344),
 (u'Two', 1301),
 (u'table', 1297),
 (u'group', 1292),
 (u'large', 1276),
 (u'person', 1204),
 (u'field', 1145),
 (u'tennis', 1064),
 (u'train', 1046),
 (u'small', 913),
 (u'near', 912),
 (u'young', 905),
 (u'black', 871),
 (u'cat', 865),
 (u'walking', 847),
 (u'two', 825),
 (u'red', 825),
 (u'dog', 790),
 (u'baseball', 785),
 (u'room', 783),
 (u'green', 777),
 (u'plate', 766),
 (u'playing', 766),
 (u'couple', 763),
 (u'parked', 695),
 (u'bathroom', 693),
 (u'front', 691),
 (u'blue', 683),
 (u'side', 666),
 (u'bus', 648),
 (u'food', 646),
 (u'filled', 639),
 (u'kitchen', 620),
 (u'flying', 603),
 (u'grass', 599),
 (u'snow', 598),
 (u'water', 597),
 (u'pizza', 593),
 (u'wooden', 591),
 (u'building', 580),
 (u'clock', 559),
 (u'laying', 556),
 (u'covered', 552),
 (u'sign', 551)